# 04 - Alternative Binders Screening

## 목표
- 저탄소 대체 결합재 빠른 스크리닝
- C2S (벨라이트), 슬래그, 비회, 메타카올린 테스트
- Ca 용출 속도 및 C-S-H 형성 지표 비교
- 유망 재료 2-3종 선별

## 대상 재료
| 재료 | CO₂ 저감 | 특징 |
|------|---------|------|
| C3S (기준) | 0% | 기존 시멘트 주성분 |
| C2S (벨라이트) | ~20% | 저온 소성, 느린 수화 |
| 슬래그 | ~80% | 철강 부산물 |
| 비회 | ~90% | 석탄 부산물 |
| 메타카올린 | ~70% | 점토 소성 |

## Phase 2-1 (빠른 스크리닝)
- 소규모 시스템 (30-50 원자)
- 짧은 시뮬레이션 (10 ps)
- 유망 재료 선별 → Phase 2-2에서 심화 연구

---
## Cell 1: Setup & Configuration

In [1]:
# ============================================================
# 설정
# ============================================================
SIMULATION_TIME = 10.0  # ps - 각 재료 스크리닝용
N_WATER = 10            # 물 분자 수
TEMPERATURE = 300       # K
SAVE_INTERVAL = 10      # steps

# Materials Project API Key (필요시 설정)
# MP_API_KEY = "your_api_key_here"
MP_API_KEY = None  # None이면 환경변수 사용
# ============================================================

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json
import time
from datetime import datetime
import warnings
from collections import Counter

# ASE
from ase import Atoms
from ase.io import read, write
from ase.io.trajectory import Trajectory
from ase.build import molecule
from ase.optimize import BFGS

# CHGNet
from chgnet.model import CHGNet
from chgnet.model.dynamics import CHGNetCalculator, MolecularDynamics
import torch

# Pymatgen & Materials Project
from pymatgen.core import Structure, Lattice
from pymatgen.io.ase import AseAtomsAdaptor

warnings.filterwarnings('ignore')

# Matplotlib settings
plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10

print("="*70)
print("04 - Alternative Binders Screening")
print("="*70)
print(f"Start: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"\nSimulation Settings:")
print(f"  Duration: {SIMULATION_TIME} ps per material")
print(f"  Water molecules: {N_WATER}")
print(f"  Temperature: {TEMPERATURE} K")
print(f"\nGPU: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device: {torch.cuda.get_device_name(0)}")

04 - Alternative Binders Screening
Start: 2026-01-29 06:41:49

Simulation Settings:
  Duration: 10.0 ps per material
  Water molecules: 10
  Temperature: 300 K

GPU: True
Device: NVIDIA GeForce RTX 4070 Laptop GPU


---
## Cell 2: Path Setup & CHGNet Load

In [2]:
# 경로 설정
WORK_DIR = Path("C:/cement_final")

FOLDERS = {
    'structures': WORK_DIR / "structures",
    'trajectories': WORK_DIR / "trajectories",
    'figures': WORK_DIR / "figures",
    'results': WORK_DIR / "results",
    'logs': WORK_DIR / "logs",
}

for folder in FOLDERS.values():
    folder.mkdir(parents=True, exist_ok=True)

print("Folders: ✓")

# CHGNet 로드
print("\nLoading CHGNet...", end=" ")
start = time.time()
chgnet = CHGNet.load()
print(f"Done ({time.time()-start:.1f}s)")

# 전역 변수
USE_GPU = torch.cuda.is_available()
DEVICE = 'cuda' if USE_GPU else 'cpu'
print(f"Device: {DEVICE}")

Folders: ✓

Loading CHGNet... CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda
Done (0.4s)
Device: cuda


---
## Cell 3: Helper Functions

In [12]:
def get_calculator():
    """CHGNet 계산기 생성"""
    return CHGNetCalculator(model=chgnet, use_device=DEVICE)

def print_structure_info(atoms, name="Structure"):
    """구조 정보 출력"""
    composition = Counter(atoms.get_chemical_symbols())
    print(f"\n{name}:")
    print(f"  Formula: {atoms.get_chemical_formula()}")
    print(f"  Atoms: {len(atoms)}")
    print(f"  Composition: {dict(composition)}")
    if atoms.cell is not None and atoms.cell.volume > 0:
        print(f"  Volume: {atoms.get_volume():.1f} Å³")

def optimize_structure(atoms, name="Structure", fmax=0.05, steps=100):
    """구조 최적화"""
    print(f"\nOptimizing {name}...")
    atoms.calc = get_calculator()
    
    initial_energy = atoms.get_potential_energy()
    
    optimizer = BFGS(atoms, logfile=None)
    start = time.time()
    optimizer.run(fmax=fmax, steps=steps)
    elapsed = time.time() - start
    
    final_energy = atoms.get_potential_energy()
    final_fmax = np.abs(atoms.get_forces()).max()
    
    print(f"  Steps: {optimizer.get_number_of_steps()}")
    print(f"  Time: {elapsed:.1f}s")
    print(f"  Energy: {initial_energy:.2f} → {final_energy:.2f} eV")
    print(f"  Fmax: {final_fmax:.4f} eV/Å {'✓' if final_fmax < fmax else '(not converged)'}")
    
    return {
        'initial_energy': initial_energy,
        'final_energy': final_energy,
        'energy_per_atom': final_energy / len(atoms),
        'fmax': final_fmax,
        'steps': optimizer.get_number_of_steps(),
        'time': elapsed
    }

def add_water(atoms, n_water=10, min_dist=2.5):
    """물 분자 추가"""
    from ase.build import molecule
    
    # 셀 확장 (물 공간 확보)
    cell = atoms.get_cell()
    new_cell = cell.copy()
    new_cell[2, 2] += 8.0  # z 방향 확장
    
    system = atoms.copy()
    system.set_cell(new_cell)
    
    # 물 분자 추가
    existing_pos = system.get_positions()
    water_template = molecule('H2O')
    
    added = 0
    max_attempts = 1000
    attempts = 0
    
    while added < n_water and attempts < max_attempts:
        attempts += 1
        
        # 무작위 위치 (상부 공간)
        x = np.random.uniform(0.5, new_cell[0, 0] - 0.5)
        y = np.random.uniform(0.5, new_cell[1, 1] - 0.5)
        z = np.random.uniform(cell[2, 2] + 1.0, new_cell[2, 2] - 1.0)
        
        # 기존 원자와 거리 확인
        pos = np.array([x, y, z])
        if len(existing_pos) > 0:
            distances = np.linalg.norm(existing_pos - pos, axis=1)
            if distances.min() < min_dist:
                continue
        
        # 물 분자 추가
        water = water_template.copy()
        water.translate(pos)
        
        # 무작위 회전
        angle = np.random.uniform(0, 360)
        water.rotate(angle, 'z', center=pos)
        
        system += water
        existing_pos = system.get_positions()
        added += 1
    
    print(f"  Added {added} water molecules")
    return system

def run_hydration_md(atoms, name, duration_ps=10.0, temperature=300, progress_interval=1.0):
    """수화 MD 시뮬레이션 (실시간 진행 상황 표시)"""
    import sys
    
    print(f"\nRunning MD for {name} ({duration_ps} ps)...")
    
    traj_file = FOLDERS['trajectories'] / f'{name}_hydration.traj'
    log_file = FOLDERS['logs'] / f'{name}_hydration.log'
    
    md = MolecularDynamics(
        atoms=atoms,
        model=chgnet,
        ensemble='nvt',
        temperature=temperature,
        timestep=1.0,
        trajectory=str(traj_file),
        logfile=str(log_file),
        loginterval=SAVE_INTERVAL,
        use_device=DEVICE
    )
    
    total_steps = int(duration_ps * 1000)
    chunk_steps = int(progress_interval * 1000)  # 진행 상황 업데이트 간격
    n_chunks = total_steps // chunk_steps
    
    start = time.time()
    completed_steps = 0
    
    print(f"  Total: {total_steps:,} steps ({duration_ps} ps)")
    print(f"  Progress: ", end="")
    
    for i in range(n_chunks):
        md.run(steps=chunk_steps)
        completed_steps += chunk_steps
        
        # 진행률 계산
        progress = completed_steps / total_steps * 100
        elapsed = time.time() - start
        speed = completed_steps / elapsed
        remaining = (total_steps - completed_steps) / speed if speed > 0 else 0
        
        # 진행 바 출력
        bar_len = 20
        filled = int(bar_len * progress / 100)
        bar = '█' * filled + '░' * (bar_len - filled)
        
        print(f"\r  Progress: [{bar}] {progress:5.1f}% | {completed_steps/1000:.1f}/{duration_ps:.1f} ps | ETA: {remaining:.0f}s  ", end="")
        sys.stdout.flush()
    
    # 남은 스텝 처리
    remaining_steps = total_steps - completed_steps
    if remaining_steps > 0:
        md.run(steps=remaining_steps)
    
    elapsed = time.time() - start
    
    print(f"\r  Progress: [{'█' * 20}] 100.0% | {duration_ps:.1f}/{duration_ps:.1f} ps | Done!       ")
    print(f"  Completed in {elapsed:.1f}s ({elapsed/60:.1f} min)")
    print(f"  Speed: {total_steps/elapsed:.0f} steps/s")
    
    return traj_file, log_file

print("Helper functions loaded ✓")

Helper functions loaded ✓


---
## Cell 4: Load C3S Reference (from previous work)

In [4]:
# C3S 참조 데이터 로드 (01-03번 노트북 결과)
print("="*70)
print("Loading C3S Reference Data")
print("="*70)

c3s_summary_file = FOLDERS['results'] / 'csh_formation_summary_10.0ps.json'

if c3s_summary_file.exists():
    with open(c3s_summary_file, 'r') as f:
        c3s_reference = json.load(f)
    
    print("\nC3S Reference (from 03번 노트북):")
    print(f"  Ca leaching rate: {c3s_reference['ca_leaching']['rate_per_ps']:.2f} Ca/ps")
    print(f"  Si coordination: {c3s_reference['si_coordination']['final_avg']:.2f}")
    print(f"  C-S-H pairs (final): {c3s_reference['csh_formation']['final_pairs']}")
else:
    print("\n⚠️ C3S reference not found - will calculate baseline")
    c3s_reference = None

# C3S 구조 로드
c3s_file = FOLDERS['structures'] / 'C3S_optimized.cif'
if c3s_file.exists():
    c3s_atoms = read(str(c3s_file))
    print_structure_info(c3s_atoms, "C3S (reference)")
else:
    print("\n⚠️ C3S_optimized.cif not found!")

Loading C3S Reference Data

C3S Reference (from 03번 노트북):
  Ca leaching rate: 0.40 Ca/ps
  Si coordination: 4.00
  C-S-H pairs (final): 8

C3S (reference):
  Formula: Ca9O15Si3
  Atoms: 27
  Composition: {'Ca': 9, 'Si': 3, 'O': 15}
  Volume: 374.5 Å³


---
## Cell 5: Create/Load Alternative Binder Structures

### 5.1 C2S (Belite) - Dicalcium Silicate

In [5]:
print("="*70)
print("Creating C2S (Belite) Structure")
print("="*70)

# C2S (β-Ca2SiO4) 구조 생성
# 단사정계 구조 근사

# 격자 파라미터 (β-C2S 근사)
a, b, c = 5.48, 6.76, 9.28  # Å
alpha, beta, gamma = 90, 94.6, 90  # degrees

lattice = Lattice.from_parameters(a, b, c, alpha, beta, gamma)

# Ca2SiO4 단위셀 (간소화)
# 실제 구조는 더 복잡하지만, CHGNet이 최적화할 것임
species = ['Ca', 'Ca', 'Si', 'O', 'O', 'O', 'O']
coords = [
    [0.25, 0.25, 0.25],  # Ca1
    [0.75, 0.75, 0.75],  # Ca2
    [0.0, 0.0, 0.0],     # Si
    [0.15, 0.15, 0.15],  # O1
    [0.85, 0.85, 0.15],  # O2
    [0.15, 0.85, 0.85],  # O3
    [0.85, 0.15, 0.85],  # O4
]

c2s_structure = Structure(lattice, species, coords)

# 2x2x1 슈퍼셀로 확장
c2s_structure.make_supercell([2, 2, 1])

# ASE Atoms로 변환
c2s_atoms = AseAtomsAdaptor.get_atoms(c2s_structure)

print_structure_info(c2s_atoms, "C2S (initial)")
print(f"  Ca/Si ratio: {sum(1 for s in c2s_atoms.get_chemical_symbols() if s == 'Ca') / sum(1 for s in c2s_atoms.get_chemical_symbols() if s == 'Si'):.2f}")

# 최적화
c2s_opt_results = optimize_structure(c2s_atoms, "C2S", fmax=0.1, steps=200)

# 저장
write(str(FOLDERS['structures'] / 'C2S_optimized.cif'), c2s_atoms)
print(f"\n✓ Saved: C2S_optimized.cif")

Creating C2S (Belite) Structure

C2S (initial):
  Formula: Ca8O16Si4
  Atoms: 28
  Composition: {'Ca': 8, 'Si': 4, 'O': 16}
  Volume: 1370.7 Å³
  Ca/Si ratio: 2.00

Optimizing C2S...
CHGNet will run on cuda
  Steps: 61
  Time: 7.6s
  Energy: -93.59 → -194.27 eV
  Fmax: 0.0658 eV/Å ✓

✓ Saved: C2S_optimized.cif


### 5.2 Slag Model (CaO-SiO₂-Al₂O₃)

In [6]:
print("="*70)
print("Creating Slag Model Structure")
print("="*70)

# 철강 슬래그 조성 (일반적)
# CaO: 35-45%, SiO2: 30-40%, Al2O3: 10-15%
# 간소화된 모델: Ca6Si4Al2O16 (무정형 근사)

# 격자
slag_cell = [10.0, 10.0, 10.0]

# 원자 배치 (무작위 초기 배치 - CHGNet이 최적화)
np.random.seed(42)  # 재현성

n_ca = 6
n_si = 4
n_al = 2
n_o = 16  # 전하 균형

positions = []
symbols = []

# Ca 배치
for i in range(n_ca):
    pos = np.random.uniform(1.0, 9.0, 3)
    positions.append(pos)
    symbols.append('Ca')

# Si 배치
for i in range(n_si):
    pos = np.random.uniform(1.0, 9.0, 3)
    positions.append(pos)
    symbols.append('Si')

# Al 배치
for i in range(n_al):
    pos = np.random.uniform(1.0, 9.0, 3)
    positions.append(pos)
    symbols.append('Al')

# O 배치 (Si, Al 근처에 배치)
si_al_pos = positions[n_ca:n_ca+n_si+n_al]
for i in range(n_o):
    # Si 또는 Al 근처에 O 배치
    center = si_al_pos[i % len(si_al_pos)]
    offset = np.random.uniform(-1.8, 1.8, 3)
    pos = center + offset
    pos = np.clip(pos, 0.5, 9.5)
    positions.append(pos)
    symbols.append('O')

slag_atoms = Atoms(
    symbols=symbols,
    positions=positions,
    cell=slag_cell,
    pbc=True
)

print_structure_info(slag_atoms, "Slag (initial - random)")

# 최적화 (더 많은 스텝 - 무작위 구조이므로)
slag_opt_results = optimize_structure(slag_atoms, "Slag", fmax=0.1, steps=300)

# 저장
write(str(FOLDERS['structures'] / 'Slag_optimized.cif'), slag_atoms)
print(f"\n✓ Saved: Slag_optimized.cif")

Creating Slag Model Structure

Slag (initial - random):
  Formula: Al2Ca6O16Si4
  Atoms: 28
  Composition: {'Ca': 6, 'Si': 4, 'Al': 2, 'O': 16}
  Volume: 1000.0 Å³

Optimizing Slag...
CHGNet will run on cuda
  Steps: 300
  Time: 37.1s
  Energy: 25.50 → -199.99 eV
  Fmax: 2.1068 eV/Å (not converged)

✓ Saved: Slag_optimized.cif


### 5.3 Fly Ash Model (SiO₂-Al₂O₃ rich)

In [7]:
print("="*70)
print("Creating Fly Ash Model Structure")
print("="*70)

# 비회 조성 (Class F)
# SiO2: 50-60%, Al2O3: 20-30%, Fe2O3: 5-15%, CaO: <10%
# 간소화된 모델: Si4Al2Ca1O12 (저Ca 포졸란)

np.random.seed(43)

fa_cell = [9.0, 9.0, 9.0]

n_si = 4
n_al = 2
n_ca = 1
n_o = 12

positions = []
symbols = []

# Si 배치 (사면체 중심)
si_positions = [
    [2.0, 2.0, 2.0],
    [6.0, 2.0, 2.0],
    [2.0, 6.0, 6.0],
    [6.0, 6.0, 6.0],
]
for pos in si_positions:
    positions.append(pos)
    symbols.append('Si')

# Al 배치
al_positions = [
    [4.0, 4.0, 2.0],
    [4.0, 4.0, 6.0],
]
for pos in al_positions:
    positions.append(pos)
    symbols.append('Al')

# Ca 배치
positions.append([4.5, 4.5, 4.5])
symbols.append('Ca')

# O 배치 (Si/Al 주위)
for i, center in enumerate(si_positions + al_positions):
    # 각 Si/Al 주위에 2개 O
    for j in range(2):
        offset = np.array([1.6, 0, 0]) if j == 0 else np.array([0, 1.6, 0])
        offset = offset * (1 if i % 2 == 0 else -1)
        pos = np.array(center) + offset + np.random.uniform(-0.2, 0.2, 3)
        pos = np.clip(pos, 0.5, 8.5)
        positions.append(pos.tolist())
        symbols.append('O')

flyash_atoms = Atoms(
    symbols=symbols,
    positions=positions,
    cell=fa_cell,
    pbc=True
)

print_structure_info(flyash_atoms, "Fly Ash (initial)")
print(f"  Si/Al ratio: {n_si/n_al:.2f}")

# 최적화
flyash_opt_results = optimize_structure(flyash_atoms, "Fly Ash", fmax=0.1, steps=300)

# 저장
write(str(FOLDERS['structures'] / 'FlyAsh_optimized.cif'), flyash_atoms)
print(f"\n✓ Saved: FlyAsh_optimized.cif")

Creating Fly Ash Model Structure

Fly Ash (initial):
  Formula: Al2CaO12Si4
  Atoms: 19
  Composition: {'Si': 4, 'Al': 2, 'Ca': 1, 'O': 12}
  Volume: 729.0 Å³
  Si/Al ratio: 2.00

Optimizing Fly Ash...
CHGNet will run on cuda
  Steps: 116
  Time: 13.3s
  Energy: -57.76 → -139.83 eV
  Fmax: 0.0937 eV/Å ✓

✓ Saved: FlyAsh_optimized.cif


### 5.4 Metakaolin Model (Al₂Si₂O₇)

In [8]:
print("="*70)
print("Creating Metakaolin Model Structure")
print("="*70)

# 메타카올린: Al2Si2O7 (탈수 카올리나이트)
# 층상 구조의 무정형화

np.random.seed(44)

mk_cell = [8.0, 8.0, 8.0]

# Al2Si2O7 × 2 = Al4Si4O14
n_al = 4
n_si = 4
n_o = 14

positions = []
symbols = []

# Al-Si 교대 배열 (층상 구조 흔적)
layer_z = [2.0, 6.0]
for z in layer_z:
    positions.append([2.0, 2.0, z])
    symbols.append('Al')
    positions.append([6.0, 2.0, z])
    symbols.append('Si')
    positions.append([2.0, 6.0, z])
    symbols.append('Si')
    positions.append([6.0, 6.0, z])
    symbols.append('Al')

# O 배치
o_count = 0
for i, (sym, pos) in enumerate(zip(symbols[:8], positions[:8])):
    if o_count >= n_o:
        break
    # 각 Al/Si 주위에 O 배치
    n_neighbors = 2 if o_count < n_o - 2 else 1
    for j in range(n_neighbors):
        if o_count >= n_o:
            break
        offset = np.random.uniform(-1.7, 1.7, 3)
        o_pos = np.array(pos) + offset
        o_pos = np.clip(o_pos, 0.5, 7.5)
        positions.append(o_pos.tolist())
        symbols.append('O')
        o_count += 1

metakaolin_atoms = Atoms(
    symbols=symbols,
    positions=positions,
    cell=mk_cell,
    pbc=True
)

print_structure_info(metakaolin_atoms, "Metakaolin (initial)")

# 최적화
mk_opt_results = optimize_structure(metakaolin_atoms, "Metakaolin", fmax=0.1, steps=300)

# 저장
write(str(FOLDERS['structures'] / 'Metakaolin_optimized.cif'), metakaolin_atoms)
print(f"\n✓ Saved: Metakaolin_optimized.cif")

Creating Metakaolin Model Structure

Metakaolin (initial):
  Formula: Al4O14Si4
  Atoms: 22
  Composition: {'Al': 4, 'Si': 4, 'O': 14}
  Volume: 512.0 Å³

Optimizing Metakaolin...
CHGNet will run on cuda
  Steps: 300
  Time: 34.5s
  Energy: -40.11 → -160.93 eV
  Fmax: 0.5606 eV/Å (not converged)

✓ Saved: Metakaolin_optimized.cif


---
## Cell 6: Summary of Optimized Structures

In [9]:
print("="*70)
print("Summary of Optimized Structures")
print("="*70)

# 모든 결과 수집
materials = {
    'C3S': {'atoms': c3s_atoms if 'c3s_atoms' in dir() else None, 'co2_reduction': 0},
    'C2S': {'atoms': c2s_atoms, 'results': c2s_opt_results, 'co2_reduction': 20},
    'Slag': {'atoms': slag_atoms, 'results': slag_opt_results, 'co2_reduction': 80},
    'FlyAsh': {'atoms': flyash_atoms, 'results': flyash_opt_results, 'co2_reduction': 90},
    'Metakaolin': {'atoms': metakaolin_atoms, 'results': mk_opt_results, 'co2_reduction': 70},
}

print("\n{:<12} {:>8} {:>12} {:>10} {:>8}".format(
    "Material", "Atoms", "E/atom (eV)", "Fmax", "CO2 ↓%"
))
print("-" * 55)

for name, data in materials.items():
    if data['atoms'] is not None:
        n_atoms = len(data['atoms'])
        if 'results' in data:
            e_per_atom = data['results']['energy_per_atom']
            fmax = data['results']['fmax']
        else:
            e_per_atom = 0
            fmax = 0
        co2 = data['co2_reduction']
        
        print(f"{name:<12} {n_atoms:>8} {e_per_atom:>12.4f} {fmax:>10.4f} {co2:>8}%")
    else:
        print(f"{name:<12} {'(reference)':>8}")

print("-" * 55)

Summary of Optimized Structures

Material        Atoms  E/atom (eV)       Fmax   CO2 ↓%
-------------------------------------------------------
C3S                27       0.0000     0.0000        0%
C2S                28      -6.9382     0.0658       20%
Slag               28      -7.1426     2.1068       80%
FlyAsh             19      -7.3593     0.0937       90%
Metakaolin         22      -7.3149     0.5606       70%
-------------------------------------------------------


---
## Cell 7: Add Water and Create Hydration Systems

In [10]:
print("="*70)
print("Creating Hydration Systems")
print("="*70)

hydration_systems = {}

for name, data in materials.items():
    if data['atoms'] is None:
        continue
    if name == 'C3S':
        # C3S는 기존 수화 시스템 사용
        c3s_hydration_file = FOLDERS['structures'] / 'C3S_hydration_initial.cif'
        if c3s_hydration_file.exists():
            hydration_systems['C3S'] = read(str(c3s_hydration_file))
            print(f"\n{name}: Loaded existing hydration system")
            print_structure_info(hydration_systems['C3S'], f"{name} + H2O")
        continue
    
    print(f"\n{name}:")
    system = add_water(data['atoms'].copy(), n_water=N_WATER)
    hydration_systems[name] = system
    print_structure_info(system, f"{name} + H2O")
    
    # 저장
    write(str(FOLDERS['structures'] / f'{name}_hydration_initial.cif'), system)
    print(f"  ✓ Saved: {name}_hydration_initial.cif")

print(f"\n✓ Created {len(hydration_systems)} hydration systems")

Creating Hydration Systems

C3S: Loaded existing hydration system

C3S + H2O:
  Formula: H10Ca9O20Si3
  Atoms: 42
  Composition: {'Ca': 9, 'Si': 3, 'O': 20, 'H': 10}
  Volume: 486.9 Å³

C2S:
  Added 10 water molecules

C2S + H2O:
  Formula: H20Ca8O26Si4
  Atoms: 58
  Composition: {'Ca': 8, 'Si': 4, 'O': 26, 'H': 20}
  Volume: 2552.3 Å³
  ✓ Saved: C2S_hydration_initial.cif

Slag:
  Added 10 water molecules

Slag + H2O:
  Formula: H20Al2Ca6O26Si4
  Atoms: 58
  Composition: {'Ca': 6, 'Si': 4, 'Al': 2, 'O': 26, 'H': 20}
  Volume: 1800.0 Å³
  ✓ Saved: Slag_hydration_initial.cif

FlyAsh:
  Added 10 water molecules

FlyAsh + H2O:
  Formula: H20Al2CaO22Si4
  Atoms: 49
  Composition: {'Si': 4, 'Al': 2, 'Ca': 1, 'O': 22, 'H': 20}
  Volume: 1377.0 Å³
  ✓ Saved: FlyAsh_hydration_initial.cif

Metakaolin:
  Added 10 water molecules

Metakaolin + H2O:
  Formula: H20Al4O24Si4
  Atoms: 52
  Composition: {'Al': 4, 'Si': 4, 'O': 24, 'H': 20}
  Volume: 1024.0 Å³
  ✓ Saved: Metakaolin_hydration_initial.cif

---
## Cell 8: Run Hydration MD Simulations

**예상 시간**: 각 재료당 ~20-30분 (10 ps, GPU)

In [14]:
print("="*70)
print("Running Hydration Simulations")
print("="*70)

md_results = {}

# C3S는 이미 완료된 결과 사용
c3s_traj = FOLDERS['trajectories'] / 'csh_formation_10.0ps.traj'
if c3s_traj.exists():
    md_results['C3S'] = {
        'traj_file': c3s_traj,
        'log_file': FOLDERS['logs'] / 'csh_formation_10.0ps.log',
        'status': 'existing'
    }
    print(f"\n✓ C3S: Using existing trajectory")

# 시뮬레이션할 재료 목록
materials_to_run = [name for name in hydration_systems.keys() if name != 'C3S']
total_materials = len(materials_to_run)

print(f"\n📋 Materials to simulate: {total_materials}")
for i, name in enumerate(materials_to_run, 1):
    print(f"   {i}. {name}")

print(f"\n⏱️  Estimated total time: ~{total_materials * 20}-{total_materials * 30} min (GPU)")
print(f"    ({SIMULATION_TIME} ps × {total_materials} materials)")

# 전체 시작 시간
total_start = time.time()

# 나머지 재료 시뮬레이션
for idx, name in enumerate(materials_to_run, 1):
    system = hydration_systems[name]
    
    print(f"\n{'='*60}")
    print(f"[{idx}/{total_materials}] Material: {name}")
    print(f"{'='*60}")
    print(f"  Atoms: {len(system)}")
    print(f"  Formula: {system.get_chemical_formula()}")
    
    try:
        mat_start = time.time()
        
        traj_file, log_file = run_hydration_md(
            system.copy(),
            name=name,
            duration_ps=SIMULATION_TIME,
            temperature=TEMPERATURE,
            progress_interval=1.0  # 1 ps마다 진행 상황 업데이트
        )
        
        mat_elapsed = time.time() - mat_start
        
        md_results[name] = {
            'traj_file': traj_file,
            'log_file': log_file,
            'status': 'completed',
            'time': mat_elapsed
        }
        
        # 남은 시간 예측
        remaining_materials = total_materials - idx
        if remaining_materials > 0:
            avg_time = (time.time() - total_start) / idx
            eta = avg_time * remaining_materials
            print(f"  📊 Remaining: {remaining_materials} materials, ETA: ~{eta/60:.1f} min")
        
    except Exception as e:
        print(f"  ⚠️ Error: {e}")
        md_results[name] = {
            'status': 'failed',
            'error': str(e)
        }

total_elapsed = time.time() - total_start

print(f"\n{'='*70}")
print("🏁 Simulation Summary")
print(f"{'='*70}")
print(f"\n  Total time: {total_elapsed:.1f}s ({total_elapsed/60:.1f} min)")
print(f"\n  Results:")
for name, result in md_results.items():
    status = result['status']
    icon = '✓' if status in ['completed', 'existing'] else '✗'
    time_str = f" ({result.get('time', 0):.0f}s)" if 'time' in result else ''
    print(f"    {icon} {name}: {status}{time_str}")

Running Hydration Simulations

✓ C3S: Using existing trajectory

📋 Materials to simulate: 4
   1. C2S
   2. Slag
   3. FlyAsh
   4. Metakaolin

⏱️  Estimated total time: ~80-120 min (GPU)
    (10.0 ps × 4 materials)

[1/4] Material: C2S
  Atoms: 58
  Formula: H20Ca8O26Si4

Running MD for C2S (10.0 ps)...
CHGNet will run on cuda
NVT-Berendsen-MD created
  Total: 10,000 steps (10.0 ps)
  Progress: [████████████████████] 100.0% | 10.0/10.0 ps | Done!       
  Completed in 911.2s (15.2 min)
  Speed: 11 steps/s
  📊 Remaining: 3 materials, ETA: ~45.6 min

[2/4] Material: Slag
  Atoms: 58
  Formula: H20Al2Ca6O26Si4

Running MD for Slag (10.0 ps)...
CHGNet will run on cuda
NVT-Berendsen-MD created
  Total: 10,000 steps (10.0 ps)
  Progress: [████████████████████] 100.0% | 10.0/10.0 ps | Done!       
  Completed in 1033.0s (17.2 min)
  Speed: 10 steps/s
  📊 Remaining: 2 materials, ETA: ~32.4 min

[3/4] Material: FlyAsh
  Atoms: 49
  Formula: H20Al2CaO22Si4

Running MD for FlyAsh (10.0 ps)...
CH

---
## Cell 9: Analysis Functions

In [15]:
def analyze_ca_leaching(traj_file, leaching_threshold=5.0):
    """Ca 용출 분석"""
    traj = Trajectory(str(traj_file))
    
    # 초기 Ca 위치
    initial = traj[0]
    symbols = initial.get_chemical_symbols()
    ca_indices = [i for i, s in enumerate(symbols) if s == 'Ca']
    
    if len(ca_indices) == 0:
        return {'n_ca': 0, 'initial_leached': 0, 'final_leached': 0, 'max_leached': 0, 'rate_per_ps': 0, 'leached_counts': []}
    
    initial_ca_pos = initial.get_positions()[ca_indices]
    initial_center = initial_ca_pos.mean(axis=0)
    
    # 시간에 따른 용출 추적
    leached_counts = []
    
    for atoms in traj:
        ca_pos = atoms.get_positions()[ca_indices]
        distances = np.linalg.norm(ca_pos - initial_center, axis=1)
        n_leached = np.sum(distances > leaching_threshold)
        leached_counts.append(n_leached)
    
    # 용출 속도 (ps당)
    duration_ps = len(traj) * 0.01  # 10 steps per 0.01 ps
    rate = (leached_counts[-1] - leached_counts[0]) / max(duration_ps, 0.1)
    
    return {
        'n_ca': len(ca_indices),
        'initial_leached': leached_counts[0],
        'final_leached': leached_counts[-1],
        'max_leached': max(leached_counts),
        'rate_per_ps': rate,
        'leached_counts': leached_counts
    }

def analyze_si_coordination(traj_file, cutoff=2.0):
    """Si 배위수 분석"""
    traj = Trajectory(str(traj_file))
    
    initial = traj[0]
    symbols = initial.get_chemical_symbols()
    si_indices = [i for i, s in enumerate(symbols) if s == 'Si']
    o_indices = [i for i, s in enumerate(symbols) if s == 'O']
    
    if len(si_indices) == 0:
        return {'n_si': 0, 'initial_cn': 0, 'final_cn': 0, 'avg_cn': 0, 'cn_evolution': []}
    
    cn_evolution = []
    
    for atoms in traj:
        positions = atoms.get_positions()
        si_pos = positions[si_indices]
        o_pos = positions[o_indices]
        
        cns = []
        for sp in si_pos:
            distances = np.linalg.norm(o_pos - sp, axis=1)
            cn = np.sum(distances < cutoff)
            cns.append(cn)
        
        cn_evolution.append(np.mean(cns))
    
    return {
        'n_si': len(si_indices),
        'initial_cn': cn_evolution[0],
        'final_cn': cn_evolution[-1],
        'avg_cn': np.mean(cn_evolution),
        'cn_evolution': cn_evolution
    }

def analyze_csh_formation(traj_file, ca_si_cutoff=3.5):
    """C-S-H 형성 지표 분석"""
    traj = Trajectory(str(traj_file))
    
    initial = traj[0]
    symbols = initial.get_chemical_symbols()
    ca_indices = [i for i, s in enumerate(symbols) if s == 'Ca']
    si_indices = [i for i, s in enumerate(symbols) if s == 'Si']
    
    if len(ca_indices) == 0 or len(si_indices) == 0:
        return {'initial_pairs': 0, 'final_pairs': 0, 'max_pairs': 0, 'min_ca_si_dist': 0, 'pairs_evolution': []}
    
    pairs_evolution = []
    min_dist_evolution = []
    
    for atoms in traj:
        positions = atoms.get_positions()
        ca_pos = positions[ca_indices]
        si_pos = positions[si_indices]
        
        # Ca-Si 거리 계산
        min_dist = float('inf')
        n_pairs = 0
        
        for cp in ca_pos:
            for sp in si_pos:
                dist = np.linalg.norm(cp - sp)
                min_dist = min(min_dist, dist)
                if dist < ca_si_cutoff:
                    n_pairs += 1
        
        pairs_evolution.append(n_pairs)
        min_dist_evolution.append(min_dist if min_dist != float('inf') else 0)
    
    return {
        'initial_pairs': pairs_evolution[0],
        'final_pairs': pairs_evolution[-1],
        'max_pairs': max(pairs_evolution),
        'min_ca_si_dist': min(min_dist_evolution),
        'pairs_evolution': pairs_evolution
    }

print("Analysis functions loaded ✓")

Analysis functions loaded ✓


---
## Cell 10: Analyze All Materials

In [16]:
print("="*70)
print("Analyzing All Materials")
print("="*70)

import sys

analysis_results = {}

# 분석할 재료 목록
materials_to_analyze = [(name, result) for name, result in md_results.items() 
                        if result['status'] in ['completed', 'existing']]
total_materials = len(materials_to_analyze)

print(f"\n📊 Materials to analyze: {total_materials}")

analysis_start = time.time()

for idx, (name, result) in enumerate(materials_to_analyze, 1):
    mat_start = time.time()
    
    print(f"\n{'-'*50}")
    print(f"[{idx}/{total_materials}] Analyzing: {name}")
    print(f"{'-'*50}")
    
    traj_file = result['traj_file']
    
    # Trajectory 크기 확인
    traj = Trajectory(str(traj_file))
    n_frames = len(traj)
    print(f"  Trajectory: {n_frames} frames")
    
    # Ca 용출
    print(f"  [1/3] Ca Leaching...", end=" ")
    sys.stdout.flush()
    ca_results = analyze_ca_leaching(traj_file)
    print(f"✓")
    print(f"        Total Ca: {ca_results['n_ca']}, Leached: {ca_results['initial_leached']} → {ca_results['final_leached']}, Rate: {ca_results['rate_per_ps']:.3f} Ca/ps")
    
    # Si 배위
    print(f"  [2/3] Si Coordination...", end=" ")
    sys.stdout.flush()
    si_results = analyze_si_coordination(traj_file)
    print(f"✓")
    print(f"        Total Si: {si_results['n_si']}, CN: {si_results['initial_cn']:.2f} → {si_results['final_cn']:.2f}")
    
    # C-S-H 형성
    print(f"  [3/3] C-S-H Formation...", end=" ")
    sys.stdout.flush()
    csh_results = analyze_csh_formation(traj_file)
    print(f"✓")
    print(f"        Pairs: {csh_results['initial_pairs']} → {csh_results['final_pairs']}, Min dist: {csh_results['min_ca_si_dist']:.2f} Å")
    
    mat_elapsed = time.time() - mat_start
    print(f"  ⏱️  Time: {mat_elapsed:.1f}s")
    
    analysis_results[name] = {
        'ca_leaching': ca_results,
        'si_coordination': si_results,
        'csh_formation': csh_results,
        'co2_reduction': materials.get(name, {}).get('co2_reduction', 0)
    }

# 실패한 재료 표시
failed_materials = [name for name, result in md_results.items() 
                    if result['status'] not in ['completed', 'existing']]
if failed_materials:
    print(f"\n⚠️ Skipped (simulation failed): {', '.join(failed_materials)}")

total_analysis_time = time.time() - analysis_start
print(f"\n{'='*70}")
print(f"✅ Analysis completed for {len(analysis_results)} materials in {total_analysis_time:.1f}s")

Analyzing All Materials

📊 Materials to analyze: 5

--------------------------------------------------
[1/5] Analyzing: C3S
--------------------------------------------------
  Trajectory: 1001 frames
  [1/3] Ca Leaching... ✓
        Total Ca: 9, Leached: 3 → 4, Rate: 0.100 Ca/ps
  [2/3] Si Coordination... ✓
        Total Si: 3, CN: 2.33 → 2.33
  [3/3] C-S-H Formation... ✓
        Pairs: 2 → 2, Min dist: 2.87 Å
  ⏱️  Time: 1.7s

--------------------------------------------------
[2/5] Analyzing: C2S
--------------------------------------------------
  Trajectory: 1001 frames
  [1/3] Ca Leaching... ✓
        Total Ca: 8, Leached: 6 → 8, Rate: 0.200 Ca/ps
  [2/3] Si Coordination... ✓
        Total Si: 4, CN: 1.25 → 1.50
  [3/3] C-S-H Formation... ✓
        Pairs: 2 → 2, Min dist: 2.73 Å
  ⏱️  Time: 1.0s

--------------------------------------------------
[3/5] Analyzing: Slag
--------------------------------------------------
  Trajectory: 1001 frames
  [1/3] Ca Leaching... ✓
        Tot

KeyError: 'initial_leached'

---
## Cell 11: Screening Score Calculation

In [ ]:
print("="*70)
print("Screening Score Calculation")
print("="*70)

# 스크리닝 점수 계산
# 가중치: CO2 저감(25%), Ca 용출(25%), Si 안정성(25%), C-S-H 형성(25%)

scores = {}

# C3S를 기준(100점)으로 정규화
c3s_data = analysis_results.get('C3S', {})

for name, data in analysis_results.items():
    score = 0
    details = {}
    
    # 1. CO2 저감 점수 (0-25점)
    co2_score = data['co2_reduction'] * 0.25
    details['co2'] = co2_score
    score += co2_score
    
    # 2. Ca 용출 점수 (활성이 좋음, 0-25점)
    ca_rate = data['ca_leaching']['rate_per_ps']
    if ca_rate > 0:
        ca_score = min(25, ca_rate / 0.5 * 25)  # 0.5 Ca/ps = 25점
    else:
        ca_score = 5  # Ca가 없는 재료도 일부 점수
    details['ca_leaching'] = ca_score
    score += ca_score
    
    # 3. Si 안정성 점수 (CN=4 유지가 좋음, 0-25점)
    si_cn = data['si_coordination']['avg_cn']
    if si_cn > 0:
        si_score = max(0, 25 - abs(si_cn - 4) * 5)
    else:
        si_score = 15  # Si가 없는 경우
    details['si_stability'] = si_score
    score += si_score
    
    # 4. C-S-H 형성 점수 (Ca-Si 쌍이 많을수록 좋음, 0-25점)
    csh_pairs = data['csh_formation']['final_pairs']
    csh_score = min(25, csh_pairs * 2.5)  # 10쌍 = 25점
    details['csh_formation'] = csh_score
    score += csh_score
    
    scores[name] = {
        'total': score,
        'details': details
    }

# 결과 출력
print("\n{:<12} {:>8} {:>8} {:>8} {:>8} {:>10}".format(
    "Material", "CO2↓", "Ca활성", "Si안정", "C-S-H", "Total"
))
print("-" * 60)

sorted_scores = sorted(scores.items(), key=lambda x: x[1]['total'], reverse=True)

for name, score_data in sorted_scores:
    d = score_data['details']
    total = score_data['total']
    print(f"{name:<12} {d['co2']:>8.1f} {d['ca_leaching']:>8.1f} {d['si_stability']:>8.1f} {d['csh_formation']:>8.1f} {total:>10.1f}")

print("-" * 60)

---
## Cell 12: Visualization - Comparison Charts

In [ ]:
print("="*70)
print("Creating Comparison Visualizations")
print("="*70)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

material_names = list(analysis_results.keys())
colors = plt.cm.Set2(np.linspace(0, 1, len(material_names)))

# 1. CO2 저감률
ax1 = axes[0, 0]
co2_values = [analysis_results[m]['co2_reduction'] for m in material_names]
bars1 = ax1.bar(material_names, co2_values, color=colors)
ax1.set_ylabel('CO₂ Reduction (%)')
ax1.set_title('(a) CO₂ Reduction Potential')
ax1.set_ylim(0, 100)
for bar, val in zip(bars1, co2_values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
             f'{val}%', ha='center', fontsize=10)

# 2. Ca 용출 속도
ax2 = axes[0, 1]
ca_rates = [analysis_results[m]['ca_leaching']['rate_per_ps'] for m in material_names]
bars2 = ax2.bar(material_names, ca_rates, color=colors)
ax2.set_ylabel('Ca Leaching Rate (Ca/ps)')
ax2.set_title('(b) Ca Leaching Activity')
for bar, val in zip(bars2, ca_rates):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
             f'{val:.2f}', ha='center', fontsize=10)

# 3. Si 배위수
ax3 = axes[1, 0]
si_cn = [analysis_results[m]['si_coordination']['avg_cn'] for m in material_names]
bars3 = ax3.bar(material_names, si_cn, color=colors)
ax3.axhline(y=4.0, color='red', linestyle='--', label='Ideal SiO₄ (CN=4)')
ax3.set_ylabel('Si Coordination Number')
ax3.set_title('(c) Si Coordination Stability')
ax3.legend()
ax3.set_ylim(0, 6)

# 4. 종합 점수
ax4 = axes[1, 1]
total_scores = [scores[m]['total'] for m in material_names]
bars4 = ax4.bar(material_names, total_scores, color=colors)
ax4.set_ylabel('Screening Score')
ax4.set_title('(d) Overall Screening Score')
ax4.set_ylim(0, 100)
for bar, val in zip(bars4, total_scores):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
             f'{val:.0f}', ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()

# 저장
fig_path = FOLDERS['figures'] / 'alternative_binders_comparison.png'
plt.savefig(fig_path, dpi=300, bbox_inches='tight')
print(f"\n✓ Saved: {fig_path}")

plt.show()

---
## Cell 13: Time Evolution Comparison

In [ ]:
print("="*70)
print("Time Evolution Comparison")
print("="*70)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 시간 축 생성 (대략적)
for name, data in analysis_results.items():
    ca_counts = data['ca_leaching'].get('leached_counts', [])
    si_cn = data['si_coordination'].get('cn_evolution', [])
    csh_pairs = data['csh_formation'].get('pairs_evolution', [])
    
    if len(ca_counts) > 0:
        time_axis = np.linspace(0, SIMULATION_TIME, len(ca_counts))
        axes[0].plot(time_axis, ca_counts, label=name, linewidth=2)
    
    if len(si_cn) > 0:
        time_axis = np.linspace(0, SIMULATION_TIME, len(si_cn))
        axes[1].plot(time_axis, si_cn, label=name, linewidth=2)
    
    if len(csh_pairs) > 0:
        time_axis = np.linspace(0, SIMULATION_TIME, len(csh_pairs))
        axes[2].plot(time_axis, csh_pairs, label=name, linewidth=2)

axes[0].set_xlabel('Time (ps)')
axes[0].set_ylabel('Leached Ca')
axes[0].set_title('Ca Leaching Evolution')
axes[0].legend()

axes[1].set_xlabel('Time (ps)')
axes[1].set_ylabel('Si Coordination Number')
axes[1].set_title('Si CN Evolution')
axes[1].axhline(y=4.0, color='gray', linestyle='--', alpha=0.5)
axes[1].legend()

axes[2].set_xlabel('Time (ps)')
axes[2].set_ylabel('Ca-Si Pairs (<3.5Å)')
axes[2].set_title('C-S-H Precursor Formation')
axes[2].legend()

plt.tight_layout()

fig_path = FOLDERS['figures'] / 'alternative_binders_evolution.png'
plt.savefig(fig_path, dpi=300, bbox_inches='tight')
print(f"\n✓ Saved: {fig_path}")

plt.show()

---
## Cell 14: Select Promising Materials

In [ ]:
print("="*70)
print("Selecting Promising Materials for Phase 2-2")
print("="*70)

# 상위 2-3개 재료 선별
n_select = 3

# C3S 제외하고 정렬 (C3S는 기준이므로)
alternative_scores = [(name, data) for name, data in sorted_scores if name != 'C3S']

print("\n📊 Selection Criteria:")
print("  - Total screening score")
print("  - CO2 reduction potential")
print("  - C-S-H formation capability")

print(f"\n🏆 Top {n_select} Promising Materials:")
print("-" * 50)

selected_materials = []

for i, (name, score_data) in enumerate(alternative_scores[:n_select]):
    print(f"\n{i+1}. {name}")
    print(f"   Total Score: {score_data['total']:.1f}/100")
    print(f"   CO2 Reduction: {analysis_results[name]['co2_reduction']}%")
    print(f"   Ca Leaching Rate: {analysis_results[name]['ca_leaching']['rate_per_ps']:.3f} Ca/ps")
    print(f"   C-S-H Pairs: {analysis_results[name]['csh_formation']['final_pairs']}")
    
    selected_materials.append(name)

print(f"\n{'='*50}")
print(f"✓ Selected for Phase 2-2: {', '.join(selected_materials)}")
print(f"{'='*50}")

---
## Cell 15: Save Results

In [ ]:
print("="*70)
print("Saving Results")
print("="*70)

# 스크리닝 결과 저장
screening_results = {
    'metadata': {
        'date': datetime.now().isoformat(),
        'simulation_time_ps': SIMULATION_TIME,
        'n_water': N_WATER,
        'temperature_K': TEMPERATURE
    },
    'materials': {},
    'scores': {},
    'selected': selected_materials
}

for name in analysis_results:
    # 분석 결과 (evolution 데이터 제외 - 너무 큼)
    screening_results['materials'][name] = {
        'co2_reduction': analysis_results[name]['co2_reduction'],
        'ca_leaching': {
            'n_ca': analysis_results[name]['ca_leaching']['n_ca'],
            'initial': analysis_results[name]['ca_leaching']['initial_leached'],
            'final': analysis_results[name]['ca_leaching']['final_leached'],
            'rate_per_ps': analysis_results[name]['ca_leaching']['rate_per_ps']
        },
        'si_coordination': {
            'n_si': analysis_results[name]['si_coordination']['n_si'],
            'avg_cn': analysis_results[name]['si_coordination']['avg_cn']
        },
        'csh_formation': {
            'initial_pairs': analysis_results[name]['csh_formation']['initial_pairs'],
            'final_pairs': analysis_results[name]['csh_formation']['final_pairs'],
            'min_ca_si_dist': analysis_results[name]['csh_formation']['min_ca_si_dist']
        }
    }
    
    # 점수
    screening_results['scores'][name] = scores[name]

# JSON 저장
results_file = FOLDERS['results'] / 'alternative_binders_screening.json'
with open(results_file, 'w') as f:
    json.dump(screening_results, f, indent=2)
print(f"\n✓ Saved: {results_file}")

# CSV 요약 저장
import csv

csv_file = FOLDERS['results'] / 'alternative_binders_summary.csv'
with open(csv_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Material', 'CO2_Reduction_%', 'Ca_Rate', 'Si_CN', 'CSH_Pairs', 'Total_Score', 'Selected'])
    
    for name in analysis_results:
        writer.writerow([
            name,
            analysis_results[name]['co2_reduction'],
            f"{analysis_results[name]['ca_leaching']['rate_per_ps']:.3f}",
            f"{analysis_results[name]['si_coordination']['avg_cn']:.2f}",
            analysis_results[name]['csh_formation']['final_pairs'],
            f"{scores[name]['total']:.1f}",
            'Yes' if name in selected_materials else 'No'
        ])

print(f"✓ Saved: {csv_file}")

---
## Cell 16: Summary & Next Steps

In [ ]:
print("="*70)
print("04 - Alternative Binders Screening: COMPLETE")
print("="*70)

print(f"\n📅 Completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print("\n📊 Summary:")
print(f"  Materials tested: {len(analysis_results)}")
print(f"  Simulation time: {SIMULATION_TIME} ps each")
print(f"  Selected for Phase 2-2: {', '.join(selected_materials)}")

print("\n📁 Generated Files:")
print("  Structures:")
for name in ['C2S', 'Slag', 'FlyAsh', 'Metakaolin']:
    print(f"    - {name}_optimized.cif")
    print(f"    - {name}_hydration_initial.cif")

print("  Trajectories:")
for name in analysis_results:
    if name != 'C3S':
        print(f"    - {name}_hydration.traj")

print("  Results:")
print("    - alternative_binders_screening.json")
print("    - alternative_binders_summary.csv")

print("  Figures:")
print("    - alternative_binders_comparison.png")
print("    - alternative_binders_evolution.png")

print("\n🎯 Next Steps (Phase 2-2):")
print("  1. 05_Extended_Simulation.ipynb")
print(f"     - Run 100 ps MD for selected materials")
print(f"     - Expand system size (150+ atoms)")
print("  2. 06_CaSi_Ratio_Study.ipynb")
print("     - Test Ca/Si ratios: 1.0, 1.5, 1.7, 2.0")

print("\n" + "="*70)
print("✅ Phase 2-1 Complete!")
print("="*70)